Libraries

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

PROJECT_ROOT = Path("..").resolve()

DATA_DIR = PROJECT_ROOT / "data" / "Converted Images"

ANNOT_DIR = PROJECT_ROOT / "data" / "Annotated Files"

OUT_DIR = PROJECT_ROOT / "outputs"
SAMPLE_DIR = OUT_DIR / "samples" / "segmentation_guided"
SAMPLE_DIR.mkdir(parents=True, exist_ok=True)

EXTS = {".jpg", ".jpeg", ".png", ".bmp"}
RANDOM_STATE = 42


FIXED_SIZE = (300, 300)  # (W, H)

def read_bgr_300(path):
    """Read color image and resize to 300x300 using bilinear interpolation (paper)."""
    bgr = cv2.imread(str(path))
    if bgr is None:
        return None
    return cv2.resize(bgr, FIXED_SIZE, interpolation=cv2.INTER_LINEAR)

def read_gray_300(path):
    """Read grayscale image and (safely) resize to 300x300."""
    gray = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if gray is None:
        return None
    return cv2.resize(gray, FIXED_SIZE, interpolation=cv2.INTER_LINEAR)

INDEX_CSV = OUT_DIR / "preprocessed_index.csv"


Test Annotated files (Normalized?)

In [2]:
from pathlib import Path

txt_path = next(ANNOT_DIR.rglob("*.txt"))
print("Sample txt:", txt_path)

with open(txt_path, "r") as f:
    line = f.readline().strip()
print("First line:", line)

parts = line.split()
nums = list(map(float, parts))
print("Parsed:", nums)

# cek apakah (xc,yc,w,h) berada di 0..1
if len(nums) == 5 and all(0.0 <= v <= 1.0 for v in nums[1:]):
    print("=> Format: YOLO normalized (aman untuk resize berapapun)")
else:
    print("=> Kemungkinan pixel coords / format lain (perlu scaling)")


Sample txt: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\data\Annotated Files\Anthracnose (1).txt
First line: 1 0.456067 0.766338 0.324639 0.331689
Parsed: [1.0, 0.456067, 0.766338, 0.324639, 0.331689]
=> Format: YOLO normalized (aman untuk resize berapapun)


Scan Dataset & Binary Labeling

In [3]:
# Load dataset from preprocessing index (keeps pipeline connected)
if not INDEX_CSV.exists():
    raise FileNotFoundError(f"Index CSV not found: {INDEX_CSV}. Run preprocess notebook first.")

df = pd.read_csv(INDEX_CSV)

# Backward-compatible column normalization
# Expected: orig_path, prep_path, class, Output
if "orig_path" not in df.columns:
    if "path" in df.columns:
        df["orig_path"] = df["path"]
    else:
        raise ValueError(f"orig_path not found. Columns: {df.columns.tolist()}")

if "class" not in df.columns:
    # try ClassName
    if "ClassName" in df.columns:
        df["class"] = df["ClassName"]
    else:
        df["class"] = "Unknown"

if "Output" not in df.columns:
    # rebuild binary label if possible
    df["Output"] = df["class"].astype(str).str.lower().apply(lambda x: 0 if x=="healthy" else 1)

print("Loaded:", INDEX_CSV)
print("Rows:", len(df))
print("Counts per class:")
display(df["class"].value_counts())

print("\nBinary counts (Output):")
display(df["Output"].value_counts())

df.head()


Loaded: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\preprocessed_index.csv
Rows: 724
Counts per class:


class
Healthy            136
Soft_Rot           129
Gray_Blight        119
Brown_Stem_Spot    119
Anthracnose        118
Stem_Canker        103
Name: count, dtype: int64


Binary counts (Output):


Output
1    588
0    136
Name: count, dtype: int64

,orig_path,prep_path,class,Output,width,height,resize_mode,gaussian,gamma,equalization
0,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,Anthracnose,1,300,300,fixed300_bilinear,"(5, 5)_sigma0",1.2,hist_eq
1,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,Anthracnose,1,300,300,fixed300_bilinear,"(5, 5)_sigma0",1.2,hist_eq
2,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,Anthracnose,1,300,300,fixed300_bilinear,"(5, 5)_sigma0",1.2,hist_eq
3,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,Anthracnose,1,300,300,fixed300_bilinear,"(5, 5)_sigma0",1.2,hist_eq
4,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,Anthracnose,1,300,300,fixed300_bilinear,"(5, 5)_sigma0",1.2,hist_eq


K-Means Segmentation

In [ ]:
def kmeans_labels_lab(bgr, k=3):
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    H, W = lab.shape[:2]
    X = lab.reshape(-1, 3).astype(np.float32)

    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init="auto")
    labels = km.fit_predict(X).reshape(H, W)
    return labels

def build_annot_index(annot_root: Path):
    idx = {}
    if not annot_root.exists():
        return idx
    for p in annot_root.rglob("*.txt"):
        idx[p.stem] = p
    return idx

ANNOT_INDEX = build_annot_index(ANNOT_DIR)
print("Annotation txt files found:", len(ANNOT_INDEX))

def find_annotation_txt(img_path: str):
    stem = Path(img_path).stem
    return ANNOT_INDEX.get(stem, None)

def parse_yolo_txt(txt_path: Path):
    boxes = []
    with open(txt_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) != 5:
                continue
            cls = int(float(parts[0]))
            xc, yc, bw, bh = map(float, parts[1:])
            boxes.append((cls, xc, yc, bw, bh))
    return boxes

def yolo_to_xyxy(box, W, H):
    _, xc, yc, bw, bh = box
    x1 = (xc - bw/2) * W
    y1 = (yc - bh/2) * H
    x2 = (xc + bw/2) * W
    y2 = (yc + bh/2) * H
    x1 = int(max(0, round(x1))); y1 = int(max(0, round(y1)))
    x2 = int(min(W-1, round(x2))); y2 = int(min(H-1, round(y2)))
    return x1, y1, x2, y2

def refine_roi_mask(mask01):
    mask = (mask01 > 0).astype(np.uint8)
    kernel = np.ones((7,7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel, iterations=1)

    n, cc, stats, _ = cv2.connectedComponentsWithStats(mask*255, connectivity=8)
    if n <= 1:
        return mask
    areas = stats[1:, cv2.CC_STAT_AREA]
    idx = 1 + int(np.argmax(areas))
    return (cc == idx).astype(np.uint8)

def lesion_mask_from_txt(shape_hw, txt_path: Path):
    H, W = shape_hw
    mask = np.zeros((H, W), dtype=np.uint8)
    boxes = parse_yolo_txt(txt_path)
    for b in boxes:
        x1, y1, x2, y2 = yolo_to_xyxy(b, W, H)
        mask[y1:y2+1, x1:x2+1] = 1
    return mask

def bbox_from_mask(mask01: np.ndarray):
    ys, xs = np.where(mask01 > 0)
    if xs.size == 0:
        return None
    x1, x2 = int(xs.min()), int(xs.max())
    y1, y2 = int(ys.min()), int(ys.max())
    return x1, y1, x2, y2

def clip_roi_to_lesion_context(roi_mask: np.ndarray, lesion_mask: np.ndarray, pad_ratio=0.35):
    """
    Clip ROI agar hanya berada di area sekitar lesi (bbox + padding).
    pad_ratio: 0.2-0.5 biasanya bagus. Makin besar = ROI makin luas.
    """
    bb = bbox_from_mask(lesion_mask)
    if bb is None:
        return roi_mask

    x1, y1, x2, y2 = bb
    H, W = roi_mask.shape

    bw = x2 - x1 + 1
    bh = y2 - y1 + 1
    pad_x = int(bw * pad_ratio)
    pad_y = int(bh * pad_ratio)

    X1 = max(0, x1 - pad_x); Y1 = max(0, y1 - pad_y)
    X2 = min(W-1, x2 + pad_x); Y2 = min(H-1, y2 + pad_y)

    clipped = np.zeros_like(roi_mask, dtype=np.uint8)
    clipped[Y1:Y2+1, X1:X2+1] = roi_mask[Y1:Y2+1, X1:X2+1]
    return clipped

def border_touch_ratio(mask01: np.ndarray) -> float:
    """berapa proporsi pixel ROI yang menyentuh border"""
    mask = (mask01 > 0).astype(np.uint8)
    H, W = mask.shape
    border = np.zeros_like(mask, dtype=np.uint8)
    border[0,:]=1; border[-1,:]=1; border[:,0]=1; border[:,-1]=1
    touch = int((mask & border).sum())
    area = int(mask.sum()) + 1e-9
    return float(touch / area)

def score_roi(labels: np.ndarray, lesion_mask: np.ndarray, chosen):
    """
    chosen: int atau list[int]
    return dict score info
    """
    if isinstance(chosen, int):
        chosen = [chosen]

    roi = np.zeros_like(labels, dtype=np.uint8)
    for kk in chosen:
        roi |= (labels == kk).astype(np.uint8)

    lesion_area = int(lesion_mask.sum())
    roi_area = int(roi.sum())
    inter = int((roi & lesion_mask).sum())

    if lesion_area == 0 or roi_area == 0 or inter == 0:
        return {"chosen": chosen, "f1": 0.0, "precision": 0.0, "recall": 0.0, "roi_area": roi_area}

    precision = inter / (roi_area + 1e-9)
    recall = inter / (lesion_area + 1e-9)

    # F1-score
    f1 = 2 * precision * recall / (precision + recall + 1e-9)

    btr = border_touch_ratio(roi)
    f1_adj = f1 * (1.0 - 0.35 * min(1.0, btr))  

    return {
        "chosen": chosen,
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1_adj),
        "roi_area": int(roi_area),
        "inter": int(inter),
        "border_touch": float(btr)
    }

def pick_roi_clusters_robust(labels: np.ndarray, lesion_mask: np.ndarray):
    """
    Coba:
    - semua single cluster
    - semua pasangan cluster (union)
    pilih yang f1 terbaik
    """
    k = int(labels.max()) + 1
    best = None

    for kk in range(k):
        info = score_roi(labels, lesion_mask, kk)
        if (best is None) or (info["f1"] > best["f1"]):
            best = info

    for a in range(k):
        for b in range(a+1, k):
            info = score_roi(labels, lesion_mask, [a, b])
            if info["f1"] > best["f1"]:
                best = info

    return best 

def segment_guided_robust(bgr, txt_path: Path, pad_ratio=0.35):
    labels = kmeans_labels_lab(bgr, k=3)
    lesion_mask = lesion_mask_from_txt(labels.shape, txt_path)

    if int(lesion_mask.sum()) == 0:
        return labels, lesion_mask, None, None

    best = pick_roi_clusters_robust(labels, lesion_mask)

    if best is None or best["f1"] <= 0:
        return labels, lesion_mask, None, None

    roi_mask = np.zeros_like(labels, dtype=np.uint8)
    for kk in best["chosen"]:
        roi_mask |= (labels == kk).astype(np.uint8)

    roi_mask = refine_roi_mask(roi_mask)

    roi_mask = clip_roi_to_lesion_context(roi_mask, lesion_mask, pad_ratio=pad_ratio)

    roi_mask = refine_roi_mask(roi_mask)

    return labels, lesion_mask, roi_mask, best






Annotation txt files found: 545


Visualization & Save Samples

In [5]:
def bgr_to_rgb(bgr):
    return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

def overlay_mask_on_bgr(bgr, mask, color=(0,0,255), alpha=0.35):
    overlay = bgr.copy()
    c = np.zeros_like(bgr)
    c[:,:,:] = np.array(color, dtype=np.uint8)
    overlay[mask==1] = (alpha*c[mask==1] + (1-alpha)*overlay[mask==1]).astype(np.uint8)
    return overlay

def prf_from_masks(roi_mask, lesion_mask):
    roi = (roi_mask > 0).astype(np.uint8)
    les = (lesion_mask > 0).astype(np.uint8)

    inter = int((roi & les).sum())
    roi_area = int(roi.sum())
    les_area = int(les.sum())

    if roi_area == 0 or les_area == 0 or inter == 0:
        return 0.0, 0.0, 0.0

    precision = inter / (roi_area + 1e-9)
    recall = inter / (les_area + 1e-9)
    f1 = 2 * precision * recall / (precision + recall + 1e-9)
    return float(precision), float(recall), float(f1)


def mask_to_3ch(mask01):
    return (mask01 * 255).astype(np.uint8)
    

sample_n = 12
sample_df = df.sample(min(sample_n, len(df)), random_state=RANDOM_STATE)

saved = 0
skipped_no_annot = 0

for i, r in enumerate(sample_df.to_dict("records"), start=1):
    img_path = r.get("orig_path", r.get("path"))
    bgr = read_bgr_300(img_path)
    if bgr is None:
        continue

    txt = find_annotation_txt(img_path)
    if txt is None:
        skipped_no_annot += 1
        continue

    labels, lesion_mask, roi_mask, best = segment_guided_robust(bgr, txt)
    if roi_mask is None:
        continue

    label_vis = (labels.astype(np.float32) / max(1, labels.max()) * 255).astype(np.uint8)
    label_vis = cv2.applyColorMap(label_vis, cv2.COLORMAP_JET)

    over_lesion = overlay_mask_on_bgr(bgr, lesion_mask, color=(0,255,255), alpha=0.35)  # yellow-ish
    over_roi    = overlay_mask_on_bgr(bgr, roi_mask,    color=(0,0,255),   alpha=0.35)  # red

    fig = plt.figure(figsize=(18, 4))
    ax1 = fig.add_subplot(1, 4, 1)
    ax2 = fig.add_subplot(1, 4, 2)
    ax3 = fig.add_subplot(1, 4, 3)
    ax4 = fig.add_subplot(1, 4, 4)

    ax1.imshow(bgr_to_rgb(bgr))
    ax1.set_title(f"BGR 300x300 - {r['class']}")
    ax1.axis("off")

    ax2.imshow(bgr_to_rgb(label_vis))
    ax2.set_title("KMeans (k=3) clusters (LAB)")
    ax2.axis("off")

    ax3.imshow(bgr_to_rgb(over_lesion))
    ax3.set_title("Annotation lesion mask (YOLO)")
    ax3.axis("off")

    ax4.imshow(bgr_to_rgb(over_roi))
    P2, R2, F12 = prf_from_masks(roi_mask, lesion_mask)
    ax4.set_title(f"ROI clusters={best['chosen']} | F1={F12:.2f} (P={P2:.2f}, R={R2:.2f})")
    ax4.axis("off")

    out_img = SAMPLE_DIR / f"seg_{saved+1:02d}_{r['class']}.png"
    fig.savefig(out_img, bbox_inches="tight")
    plt.close(fig)
    saved += 1

print("Saved samples:", saved)
print("Skipped (no annotation found):", skipped_no_annot)
print("Output folder:", SAMPLE_DIR)


c:\Users\User\.conda\envs\pengenalan-pola\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


Saved samples: 7
Skipped (no annotation found): 5
Output folder: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\samples\segmentation_guided
